In [2]:
#Code to download all needed libraries - needs to be run every time

from __future__ import division #used in Python2.x so the division btw integers result in float, default in python3
from pyomo.environ import * #library for modeling and optimisation
import argparse
from pyomo.opt import SolverStatus, TerminationCondition #provide status of solvers
import pandas as pd #data manipulation library
import numpy as np 
import pickle
import random
import sqlite3
import copy
import random
import csv

import warnings

import os
warnings.filterwarnings('ignore')


INTEREST = 0.03 #interest rate



path =  #specify the path to the working folder

In [12]:
#The main optimization model - single objective at this point


class optimization:
    def __init__(self):
        
        data_opt =  pd.read_csv(path+"2025-01-29_cells_all_rev_hsi.csv") #update input file
        all_data = data_opt
        all_data = all_data.set_index(['stand','schedule','period']) #reassigns the Df with new multi-level index
        
        all_data = all_data.fillna(0) #fills any missing values with 0
        all_data['period'] = all_data.index.get_level_values(2)

        
        self.data_opt=all_data #store the fetched or preprocessed data for use within the class
        self.combinations = 1
        self.all_data = self.data_opt
        self.Index_values = self.all_data.drop(['period'], axis=1).reset_index().set_index(['stand','schedule']).index.unique() #we drop year, because we dont make decision for each year
        self.area = self.all_data.loc[slice(None),1,1]['area']
        self.all_data = self.all_data.fillna(0)

        self.createModel()

    def createModel(self):
        
        # Define sets, variables, and constraints for the optimization problem
        
        self.model1 = ConcreteModel() #defining the model
        
        self.model1.stands = Set(initialize = list(set(self.all_data.index.get_level_values(0)))) #initialized with unique values from the first level of the index created earlier
        self.model1.year = Set(initialize = list(set(self.all_data.index.get_level_values(2)))) #initialized with unique values from the third level of the index created earlier
               
        self.model1.regimes = Set(initialize = list(set(self.all_data.index.get_level_values(1))))
        self.model1.scen_index = Set(initialize= [i for i in range(0,self.combinations)])
        self.model1.Index_values = self.Index_values

        

        def index_rule(model1):
            index = []
            for (s,r) in model1.Index_values: 
                index.append((s,r))
            return index
        self.model1.index1 = Set(dimen=2, initialize=index_rule)

        #Decision variable
        self.model1.X1 = Var(self.model1.index1, within=Boolean, initialize=1) 

        self.all_data['period'] = self.all_data.index.get_level_values(2)

        #Objective and constraint don't need to be adjusted here, leave as it is, used to create the optimization model in the code. 
        #objective function (NPV in this case):
        self.model1.NPV= Var(within=NonNegativeReals) 
        def NPV_INVENTORY(model1):
            row_sum = sum(self.all_data.npv.loc[(s,r,6)]*self.all_data.area.loc[(s,r,6)]* self.model1.X1[(s,r)]  for (s,r) in self.model1.index1)
            return self.model1.NPV ==row_sum
        self.model1.NPV_INV= Constraint(rule=NPV_INVENTORY) 
        
        def outcome_rule(model1):
            return self.model1.NPV 
        self.model1.OBJ = Objective(rule=outcome_rule, sense=maximize)

        #Constraint that ensures that each stand (cell) is prescribed only one management regime
        def regime_rule(model1, s):
            row_sum = sum(model1.X1[(s,r)] for r in [x[1] for x in model1.index1 if x[0] == s])
            return row_sum == 1
        self.model1.regime_limit = Constraint(self.model1.stands, rule=regime_rule)

        # ============================
        # Clustering constraints
        # ============================
        group_path = r"S:\Data_for_optimization\grouping_options_6.pkl"  #Change this to path to the dictionary with clustering information

        # Load the dictionary from the pickle file
        with open(group_path, "rb") as f:
            grouping_options = pickle.load(f)

        self.model1.CL = Set(initialize=grouping_options.keys())  #CL is the keys of the dictionary - Clusters
        
        #Define regimes per CL dynamically from the dictionary
        self.model1.segment_dict = {}
        for cl in self.model1.CL:
            self.model1.segment_dict[cl] = list(grouping_options[cl].keys())

        
        self.model1.z = Var(self.model1.CL, within=Boolean)

        self.model1.all_segments = [(cl, seg) for cl in self.model1.CL for seg in self.model1.segment_dict[cl]]
        self.model1.y = Var(self.model1.all_segments, self.model1.regimes, within=Boolean)
        
        
        #Constraint for selecting exactly one cluster (z)
        def cluster_selection_rule(model1):
            return sum(model1.z[c] for c in model1.CL) == 1

        self.model1.cluster_selection = Constraint(rule=cluster_selection_rule)

        def cluster_segment_assignment_rule(model1, c, r):
            #Sum of y[c, r, j] over all regimes (j) for segment r in cluster c
            sum_y = sum(model1.y[c, r, j] for j in model1.regimes)
            
            #Constraint: the sum should equal 1, meaning exactly one regime is assigned to segment r in cluster c
            #This constraint should only apply if z[c] == 1
            return sum_y == model1.z[c]  #Only when z[c] == 1, the sum_y == 1 is enforced

        #Apply the constraint for each (cl, segment) pair in self.model1.all_segments
        self.model1.cluster_segment_assignment = Constraint(self.model1.all_segments, rule=cluster_segment_assignment_rule)

        #Create a dictionary mapping each segment to the list of stands in that segment
        self.model1.stands_in_segment = {
            r: [i for c in grouping_options.values() for i in c.get(r, [])]
            for r in set(key for cl in grouping_options.values() for key in cl.keys())
        }

        #Convert it to a Pyomo Set for compatibility
        self.model1.stands_in_segment_set = Set(initialize=self.model1.stands_in_segment.keys())
        
        def clustering_consistency_rule(model1, c, r, j):
            #Ensure sum of y_crj over all clusters covering segment r matches the decision variable X1[i, j]
            return model1.y[c, r, j] * len(grouping_options[c][r]) <= sum(model1.X1[i, j] for i in grouping_options[c][r])

        #Apply the constraint over valid (cluster, segment) pairs
        self.model1.clustering_consistency = Constraint(
            self.model1.all_segments,  
            self.model1.regimes,  
            rule=clustering_consistency_rule
        )


        #Constraints for income slack variables
        
        self.model1.Income = Var(self.model1.year, within=NonNegativeReals) #variables for periodic incomes

        def Income_periodic(model1, t):
            row_sum = sum(self.all_data.INC.loc[(s,r,t)]*self.all_data.area.loc[(s,r,t)]* self.model1.X1[(s,r)]  for (s,r) in self.model1.index1)
            return self.model1.Income[t] == row_sum

        self.model1.Income_period = Constraint(self.model1.year, rule=Income_periodic)

        self.model1.target_income = Param(default=400000, mutable=True)
        # Create a slack variable for each period (nonnegative)
        self.model1.slack = Var(self.model1.year, within=NonNegativeReals)

        def income_target_rule(model1, t):
            # Income[t] plus slack must meet or exceed target_income.
            return model1.Income[t] + model1.slack[t] >= self.model1.target_income
        self.model1.income_target = Constraint(self.model1.year, rule=income_target_rule)

        self.model1.TotalSlack = Var(within=NonNegativeReals, initialize=0)
        # Define the constraint to compute the total slack sum
        def total_slack_constraint(model1):
            return model1.TotalSlack == sum(model1.slack[k] for k in model1.year)
        self.model1.TotalSlackConstraint = Constraint(rule=total_slack_constraint)
        print("Slack constraint")

        #constraints for CHSI variables
        self.all_data['CHSI'] = 1- ((1-self.all_data.LSWP)*(1-self.all_data.TTWP)*(1-self.all_data.LTT)*(1-self.all_data.CAP)*(1-self.all_data.HAZ))


        self.model1.HSI=Var(self.model1.stands, self.model1.year, within=NonNegativeReals)
        def COMBINED_HSI(model1, s, k):
            birds = (sum(self.model1.X1[(s,r)] * self.all_data.CHSI.loc[(s,r,k)] for r in [x[1] for x in self.model1.index1 if x[0] == s] ) )
            return self.model1.HSI[(s,k)] == birds
        self.model1.COMB_HSI = Constraint(self.model1.stands,self.model1.year, rule=COMBINED_HSI)

        #A function that evaluates the combined HSI for the landscape
        self.model1.Landscape_HSI=Var(self.model1.year, within=NonNegativeReals)
        def Landscape_HSI(model1, k):
            Land_HSI = sum(self.model1.HSI[(s,k)]*self.area[s] for s in self.model1.stands)
            return self.model1.Landscape_HSI[k] == Land_HSI
        self.model1.LAND_HSI = Constraint(self.model1.year, rule=Landscape_HSI)

        
        self.model1.Landscape_HSI_Tot = Var(within=NonNegativeReals)
        def HSI_Tot(model1):
            row_sum = sum(self.model1.Landscape_HSI[k] for k in self.model1.year)
            return self.model1.Landscape_HSI_Tot == row_sum
        self.model1.LAND_HSI_TOTAL = Constraint(rule=HSI_Tot)

        #Equations to compute standing end volume 

        self.model1.EndVolume = Var(within=NonNegativeReals)

        def End_volume_constraint(model1):
            row_sum = sum(
                (self.all_data.V_Pine_end.loc[(s, r, 10)] + 
                self.all_data.V_Spruce_end.loc[(s, r, 10)] + 
                self.all_data.V_Birch_end.loc[(s, r, 10)]) * 
                self.all_data.area.loc[(s, r, 10)] * self.model1.X1[(s, r)]
                for (s, r) in self.model1.index1
            )
            return self.model1.EndVolume == row_sum
        self.model1.EndInv = Constraint(rule=End_volume_constraint)




    def solve(self):
        # Specify the solver and solve the model
        opt = SolverFactory('cbc') 
        #opt.options['mipgap'] = mip_gap  # Set the relative MIP gap
        self.results = opt.solve(self.model1,tee=True) #We solve a problem, but do not show the solver output

# Create an optimization object        
t1 = optimization()

t1.solve()

#Function to extract decision variables from the optimized model
def GET_DECISION_DATA():
        st = []
        reg = []
        vals = []
        for (s,r) in t1.model1.index1:
            st = st+[s]
            reg = reg+[r] 
            vals = vals+[t1.model1.X1[(s,r)].value]
        data = {"stand":st,"schedule":reg,"value":vals}
        df= pd.DataFrame(data)
        df = df.set_index(['stand','schedule'])
        return df

# Extract decision data and merge with original dataset    
dec  = GET_DECISION_DATA()
merged_df = dec.merge(t1.all_data, left_index=True, right_index=True, how='left')

#merged_df.to_csv(path+"Data/Outputs/2025-02-07_cells_segments_test.csv")

Slack constraint

Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 22.1.0.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2022.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile 'D:\NMBU\TEMP\tmpfbzgamr7.cplex.log' open.
CPLEX> New value for mixed integer optimality gap tolerance: 0.05
CPLEX> Problem 'D:\NMBU\TEMP\tmpb8xnrfsb.pyomo.lp' read.
Read time = 1.00 sec. (55.17 ticks)
CPLEX> Problem name         : D:\NMBU\TEMP\tmpb8xnrfsb.pyomo.lp
Objective sense      : Maximize
Variables            :  231965  [Nneg: 1904,  Binary: 230061]
Objective nonzeros   :       1
Linear constraints   :  207140  [Less: 203604,  Greater: 10,  Equal: 3526]
  Nonzeros           : 3010582
  RHS nonzeros       :     198

Variables            : Min LB: 0.000000         Max UB:

In [22]:
#Print values for main criteria


value(t1.model1.OBJ)



print("NPV:")
print(t1.model1.NPV.value)
print("Total slack:")
print(t1.model1.TotalSlack.value)
print("EndVolume:")
print(t1.model1.EndVolume.value)
print("Landscape_HSI_Tot:")
print(t1.model1.Landscape_HSI_Tot.value)



NPV:
122959.60462680818
Total slack:
4619411.037442406
EndVolume:
646.1840378091389
Landscape_HSI_Tot:
6.693822671229055


In [21]:
#Transform problem into multi-objective
#Update min and max values here from payofftables generated when running cells alone 


t1.model1.NPV_min = Param(default=1375007.80, mutable=True)
t1.model1.NPV_max = Param(default=2371796.30, mutable=True)


t1.model1.TotalSlack_min = Param(default=0, mutable=True)
t1.model1.TotalSlack_max = Param(default=2400000.00, mutable=True)


t1.model1.HSI_min = Param(default=4.85, mutable=True)
t1.model1.HSI_max = Param(default=171.43, mutable=True)


t1.model1.EndVol_min = Param(default=297.30, mutable=True)
t1.model1.EndVol_max = Param(default=10641.58, mutable=True)

#Update targets for each criteria

t1.model1.NPV_target = Param(default=2000000, mutable=True)
t1.model1.EndVol_target = Param(default=8000, mutable=True)
t1.model1.TotalSlack_target = Param(default=100000, mutable=True)
t1.model1.HSI_target = Param(default=150, mutable=True)

t1.model1.D = Var(within=Reals)

def NPV_D_constraint(model1):
    row_sum = ((t1.model1.NPV - t1.model1.NPV_target)/(t1.model1.NPV_max-t1.model1.NPV_min))
    return t1.model1.D <= row_sum 
t1.model1.NPV_D = Constraint(rule=NPV_D_constraint)
 
def TotalSlack_D_constraint(model1):
    row_sum = ((t1.model1.TotalSlack_target-t1.model1.TotalSlack)/(t1.model1.TotalSlack_max - t1.model1.TotalSlack_min))
    return t1.model1.D <= row_sum 
t1.model1.TotalSlack_D = Constraint(rule=TotalSlack_D_constraint)

def HSI_D_constraint(model1):
    row_sum = ((t1.model1.Landscape_HSI_Tot - t1.model1.HSI_target)/(t1.model1.HSI_max - t1.model1.HSI_min))
    return t1.model1.D <= row_sum 
t1.model1.HSI_D = Constraint(rule=HSI_D_constraint)

def End_V_D_constraint(model1):
    row_sum = ((t1.model1.EndVolume - t1.model1.EndVol_target)/(t1.model1.EndVol_max - t1.model1.EndVol_min))
    return t1.model1.D <= row_sum 
t1.model1.END_V_D = Constraint(rule=End_V_D_constraint)


#Updated objective function

def outcome_rule(model1):

    
    return ((t1.model1.D + 0.001*(((t1.model1.NPV)/(t1.model1.NPV_max-t1.model1.NPV_min)) + 
            ((t1.model1.EndVolume)/(t1.model1.EndVol_max - t1.model1.EndVol_min)) + 
            ((t1.model1.TotalSlack)/(t1.model1.TotalSlack_max - t1.model1.TotalSlack_min)) +
            ((t1.model1.Landscape_HSI_Tot)/(t1.model1.HSI_max - t1.model1.HSI_min)) 
            
            ))*10000) 
t1.model1.OBJ = Objective(rule=outcome_rule, sense=maximize)

t1.solve()

#Function to extract decision variables from the optimized model
def GET_DECISION_DATA():
        st = []
        reg = []
        vals = []
        for (s,r) in t1.model1.index1:
            st = st+[s]
            reg = reg+[r] 
            vals = vals+[t1.model1.X1[(s,r)].value]
        data = {"stand":st,"schedule":reg,"value":vals}
        df= pd.DataFrame(data)
        df = df.set_index(['stand','schedule'])
        return df

# Extract decision data and merge with original dataset    
dec  = GET_DECISION_DATA()
merged_df = dec.merge(t1.all_data, left_index=True, right_index=True, how='left')

#merged_df.to_csv(path+"Data/Outputs/2025-02-07_cells_segments_test.csv")

'pyomo.core.base.param.ScalarParam'>) on block unknown with a new Component
(type=<class 'pyomo.core.base.param.ScalarParam'>). This is usually indicative
of a modelling error. To avoid this warning, use block.del_component() and
block.add_component().
'pyomo.core.base.param.ScalarParam'>) on block unknown with a new Component
(type=<class 'pyomo.core.base.param.ScalarParam'>). This is usually indicative
of a modelling error. To avoid this warning, use block.del_component() and
block.add_component().
(type=<class 'pyomo.core.base.param.ScalarParam'>) on block unknown with a new
Component (type=<class 'pyomo.core.base.param.ScalarParam'>). This is usually
indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().
(type=<class 'pyomo.core.base.param.ScalarParam'>) on block unknown with a new
Component (type=<class 'pyomo.core.base.param.ScalarParam'>). This is usually
indicative of a modelling error. To avoid this warning, use
block.del_c

In [23]:
#Code to print non-zero y, x, and z variables 

print("y Variables:")
for c in t1.model1.CL:
    for r in t1.model1.segment_dict[c]:
        for j in t1.model1.regimes:
            if t1.model1.y[c, r, j].value > 0:
                print(f"y[{c}, {r}, {j}] = {t1.model1.y[c, r, j].value}")

print("x Variables:")
for (i, j) in t1.model1.index1:
    
    if (t1.model1.X1[i, j].value > 0):
        print(f"x[{i}, {j}] = {t1.model1.X1[i, j].value}")


print("z Variables:")
for c in t1.model1.CL:
    if t1.model1.z[c].value > 0:
                print(f"z[{c}] = {t1.model1.z[c].value}")

value(t1.model1.OBJ)


y Variables:
y[80, 0, 125.0] = 1.0
y[80, 1, 130.0] = 1.0
y[80, 2, 126.0] = 1.0
y[80, 3, 124.0] = 1.0
y[80, 4, 132.0] = 1.0
y[80, 5, 2.0] = 1.0
y[80, 6, 118.0] = 1.0
y[80, 7, 128.0] = 1.0
y[80, 8, 120.0] = 1.0
y[80, 11, 122.0] = 1.0
y[80, 12, 117.0] = 1.0
y[80, 13, 117.0] = 1.0
y[80, 14, 126.0] = 1.0
y[80, 15, 54.0] = 1.0
y[80, 16, 82.0] = 1.0
y[80, 17, 57.0] = 1.0
y[80, 18, 127.0] = 1.0
y[80, 21, 126.0] = 1.0
y[80, 22, 68.0] = 1.0
y[80, 26, 126.0] = 1.0
x Variables:
x[373157.0, 126.0] = 1.0
x[373094.0, 126.0] = 1.0
x[373037.0, 126.0] = 1.0
x[372984.0, 126.0] = 1.0
x[372926.0, 126.0] = 1.0
x[372868.0, 126.0] = 1.0
x[372811.0, 126.0] = 1.0
x[372759.0, 124.0] = 1.0
x[372703.0, 127.0] = 1.0
x[372652.0, 127.0] = 1.0
x[372594.0, 127.0] = 1.0
x[373159.0, 126.0] = 1.0
x[373095.0, 126.0] = 1.0
x[373038.0, 126.0] = 1.0
x[372985.0, 126.0] = 1.0
x[372927.0, 126.0] = 1.0
x[372869.0, 126.0] = 1.0
x[372812.0, 124.0] = 1.0
x[372760.0, 124.0] = 1.0
x[372704.0, 127.0] = 1.0
x[372653.0, 127.0] = 1.0
x[37

-18809.371703118446